In [1]:
!pip3 install ktrain

In [2]:
import ktrain
from ktrain import text

ktrain.__version__

'0.26.2'

In [3]:
categories = ['sci.crypt', 'sci.electronics',
             'talk.politics.misc', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
df = fetch_20newsgroups(subset='all', categories=categories, shuffle=True, random_state=42)

print('size of data: %s' % (len(df['data'])))

size of data: 3740


In [4]:
X = df.data
y = df.target

from sklearn.model_selection import train_test_split
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size = 0.16)

len(X_train) # 60, 20, 20 split

2513

In [5]:
df.target_names

['sci.crypt', 'sci.electronics', 'sci.med', 'talk.politics.misc']

In [6]:
import pickle

preprocessed_data = (X_train, y_train, X_val, y_val, X_test, y_test)

pickle_out = open("preprocessed_data","wb")
pickle.dump(preprocessed_data, pickle_out)
pickle_out.close()
print('done')

done


In [7]:
import pickle
pickle_in = open("preprocessed_data","rb")
preprocessed_data = pickle.load(pickle_in)
X_train, y_train, X_val, y_val, X_test, y_test = preprocessed_data

In [8]:
# step 1 create a transformer instance
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, classes=df.target_names)

# step 2 preprocess data
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_val, y_val)
test = t.preprocess_test(X_test, y_test)

# step 3 create a model and wrap in learner
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:414: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')



preprocessing train...
language: en
train sequence lengths:
	mean : 311
	95percentile : 780
	99percentile : 2412


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 295
	95percentile : 794
	99percentile : 2223


preprocessing test...
language: en
test sequence lengths:
	mean : 328
	95percentile : 800
	99percentile : 3587


In [9]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
419/419 [==============================] - 172s 388ms/step - loss: 1.1244 - accuracy: 0.5656 - val_loss: 0.1831 - val_accuracy: 0.9603
Epoch 2/4
419/419 [==============================] - 165s 393ms/step - loss: 0.1380 - accuracy: 0.9679 - val_loss: 0.1093 - val_accuracy: 0.9666
Epoch 3/4
419/419 [==============================] - 165s 393ms/step - loss: 0.0532 - accuracy: 0.9854 - val_loss: 0.1012 - val_accuracy: 0.9708
Epoch 4/4
419/419 [==============================] - 165s 393ms/step - loss: 0.0283 - accuracy: 0.9953 - val_loss: 0.0956 - val_accuracy: 0.9729


In [10]:
# step 7 make predictions on new data
predictor = ktrain.get_predictor(learner.model, preproc=t)
predictor.predict('Jesus Christ is the central figure of Christianity.')

'talk.politics.misc'

In [13]:
import pandas
url = 'https://raw.githubusercontent.com/RamblerInvestmentFund/Automated-Sentiment-Analysis/main/Sentiment_Analysis/sample.csv'
col_list = ["timestamp", " article title"]
dr = pandas.read_csv(url, usecols=col_list)

predictor = ktrain.get_predictor(learner.model, preproc=t)
for i in dr[' article title']:
  print(predictor.predict(i))

sci.med
talk.politics.misc
sci.electronics


In [15]:
!pip3 install git+https://github.com/amaiya/eli5@tfkeras_0_10_1

  Cloning https://github.com/amaiya/eli5 (to revision tfkeras_0_10_1) to /tmp/pip-req-build-mj_b34nv
  Running command git clone -q https://github.com/amaiya/eli5 /tmp/pip-req-build-mj_b34nv
  Running command git checkout -b tfkeras_0_10_1 --track origin/tfkeras_0_10_1
  Switched to a new branch 'tfkeras_0_10_1'
  Branch 'tfkeras_0_10_1' set up to track remote branch 'tfkeras_0_10_1' from 'origin'.
  Created wheel for eli5: filename=eli5-0.10.1-py2.py3-none-any.whl size=106832 sha256=774f21a4956e4a8f926ed605a8c402b459c341dcd8abf18744c1ca4d196a7d75
  Stored in directory: /tmp/pip-ephem-wheel-cache-wafvnn7p/wheels/51/59/0a/0f48442b8d209583a4453580938d7ba2270aca40edacee6d45
Successfully built eli5


In [ ]:
y_pred_raw = predictor.predict(X_test)

In [ ]:
classes = predictor.get_classes()

classes.index(y_pred_raw[0])

In [ ]:
classes = predictor.get_classes()

y_pred = []
for i in range(len(y_pred_raw)):
  sample = y_pred_raw[i]
  y_pred.append(classes.index(sample))

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)

acc

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

cm

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))